In [1]:
import math
import torch
import numpy as np
from typing import Union
import pandas as pd
from sentence_transformers import SentenceTransformer

C:\Users\BracklyMurunga\Desktop\AI SUNDAYS\PlayGenie\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\BracklyMurunga\Desktop\AI SUNDAYS\PlayGenie\src\playgenie\data\dataset.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(process_text)
C:\Users\BracklyMurunga\Desktop\AI SUNDAYS\PlayGenie\src\playgenie\data\dataset.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
        df = df.dropna()

        def process_text(text: str) -> Union[str, np.nan]:
            try:
                return str(text).strip().lower()
            except Exception:
                return np.nan

        for col in df.columns:
            df[col] = df[col].apply(process_text)
        df['trackname'] = df['trackname'].str.cat(df['artistname'], sep=' by ')
        df = df.groupby(by="playlistname").agg(list).reset_index().drop(columns=['playlistname'])
        df = df[[len(x) > 40 for x in df['trackname']]]
        return df[['trackname']].reset_index()

In [ ]:
df = pd.read_csv("../data/spotify_dataset.csv",on_bad_lines='skip')
sentence_encoder = SentenceTransformer("all-MiniLM-L6-v2")
df = df.rename(columns={x: x.strip().replace('"', '') for x in df.columns})

songs_to_index = {x: i for i, x in enumerate(df['trackname'].unique())}
playlist_to_index = {x: i for i, x in enumerate(df['playlistname'].unique())}

df = preprocess_df(df)

In [ ]:
from tqdm import tqdm

train_data = []
val_data = []
test_data = []

for _,row in tqdm(df.iterrows()):
    context = 11
    splits = len(row['trackname'])/context

    playlist = np.array_split(np.array(row['trackname']),splits)
    train_idx = math.floor(splits*0.6)
    val_idx = math.floor(splits-1)

    train_data = train_data+[sentence_encoder.encode(p) for p in playlist[:train_idx]]
    val_data = val_data+[sentence_encoder.encode(p) for p in playlist[train_idx:val_idx]]
    test_data = test_data+[sentence_encoder.encode(p) for p in playlist[val_idx:]]

train_data = torch.tensor(train_data)
val_data = torch.tensor(val_data)
test_data = torch.tensor(test_data)

3286it [1:15:03,  1.85it/s]

In [59]:
from safetensors.torch import save_file

tensors = {
    "train": torch.tensor(train_data),
    "validation": torch.tensor(val_data),
    "test": torch.tensor(test_data)
}
save_file(tensors, "spotify_dataset.safetensors")

C:\Users\BracklyMurunga\AppData\Local\Temp\ipykernel_27616\100074661.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "train": torch.tensor(train_data),
C:\Users\BracklyMurunga\AppData\Local\Temp\ipykernel_27616\100074661.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "validation": torch.tensor(val_data),
C:\Users\BracklyMurunga\AppData\Local\Temp\ipykernel_27616\100074661.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "test": torch.tensor(test_data)
